## Food-101 整理出小测试集

In [3]:
from pathlib import Path
import os
root = Path('/www/datasets/') / 'Ingredients101'
# image_dir = root / 'ready_chinese_food'
label_dir = root / 'Annotations'
num_classes = 101
ratio = 0.9
for split in ['train', 'test', 'val']:
    label_file = label_dir / f"{split}_labels.txt"
    image_file = label_dir / f"{split}_images.txt"

    all_labels = {}
    all_images = {}
    num_data = 0

    all_images_list = []
    with open(image_file, 'r') as f:
        for line in f.readlines():
            _line = line.rstrip()
            all_images_list.append(_line)

    all_labels_list = []
    with open(label_file, 'r') as f:
        for line in f.readlines():
            _line = line.rstrip()
            all_labels_list.append(_line)

    with open(label_file, 'r') as f:
        for line in f.readlines():
            _line = line.rstrip()
            if _line not in all_labels:
                all_labels[_line] = []
                all_images[_line] = []
            all_images[_line].append(all_images_list[num_data])
            #  |
            #  V
            all_labels[_line].append(all_labels_list[num_data])
            num_data += 1
    
    print(f'{split} origin len:', num_data, '|', len(all_labels.keys()), len(all_images.keys()), num_classes)
    assert len(all_labels.keys()) == len(all_images.keys()) == num_classes
    
    num = int(num_data * ratio)
    
    labeled_images = []
    unlabeled_images = []
    labeled_labels = []
    unlabeled_labels = []
    threshold = num // num_classes
    for _label in all_labels.keys():
        labeled_images.extend(all_images[_label][:threshold])
        unlabeled_images.extend(all_images[_label][threshold:])

        labeled_labels.extend(all_labels[_label][:threshold])
        unlabeled_labels.extend(all_labels[_label][threshold:])

    # print('==> split:', split, 'ratio:', ratio, 'num:', num, 'threshold:', threshold, len(labeled_images))
    assert len(labeled_images) == len(labeled_labels)
    assert len(unlabeled_images) == len(unlabeled_labels)
    print(f'{split} labeled len:', len(labeled_images))
    print(f'{split} unlabeled len:', len(unlabeled_images))
    print('----------------')
    
    labeled_image_filename = f'labeled_{int(ratio*100)}_{split}_images.txt'
    unlabeled_image_filename = f'unlabeled_{int(ratio*100)}_{split}_images.txt'

    # save to txt file
    with open(label_dir / labeled_image_filename, 'w') as f:
        for image in labeled_images:
            f.write(str(image) + '\n')

    with open(label_dir / unlabeled_image_filename, 'w') as f:
        for image in unlabeled_images:
            f.write(str(image) + '\n')

    labeled_label_filename = f'labeled_{int(ratio*100)}_{split}_labels.txt'
    unlabeled_label_filename = f'unlabeled_{int(ratio*100)}_{split}_labels.txt'

    # save to txt file
    with open(label_dir / labeled_label_filename, 'w') as f:
        for label in labeled_labels:
            f.write(str(label) + '\n')

    with open(label_dir / unlabeled_label_filename, 'w') as f:
        for label in unlabeled_labels:
            f.write(str(label) + '\n')
            


train origin len: 68175 | 101 101 101
train labeled len: 61307
train unlabeled len: 6868
----------------
test origin len: 25250 | 101 101 101
test labeled len: 22725
test unlabeled len: 2525
----------------
val origin len: 7575 | 101 101 101
val labeled len: 6767
val unlabeled len: 808
----------------


## 标签统计

In [ ]:
from pprint import pprint
from create_dataset import Food172

In [ ]:

food172 = Food172('/www/datasets/Food172', sub_labels=[0, 308])
image_ingredient_label_map = food172.image_ingredient_label_map

# ingredient_label_names
# ingredient_label_names = food172.ingredient_label_names
print(len(image_ingredient_label_map))

In [ ]:
ingredient_label_names = food172.ingredient_label_names
print(ingredient_label_names)

In [ ]:
# 统计每个食材出现的次数
ingredient_count = {}
for image_name, ingredient_label in image_ingredient_label_map.items():
    for i, label in enumerate(ingredient_label):
        if label == 1:
            if i not in ingredient_count:
                ingredient_count[i] = 1
            else:
                ingredient_count[i] += 1
# pprint(ingredient_count)

In [ ]:
sorted_ingredient_count = sorted(ingredient_count.items(), key=lambda x: x[1], reverse=True)
pprint(sorted_ingredient_count[:1])
pprint(sorted_ingredient_count[-1:])
# ingredient_name_count
ingredient_name_count = {}
for i, count in ingredient_count.items():
    ingredient_name_count[ingredient_label_names[i]] = count

# print ingredient_name_count with top 10
sorted_ingredient_name_count = sorted(ingredient_name_count.items(), key=lambda x: x[1], reverse=True)
# pprint(sorted_ingredient_name_count[:10])

In [ ]:
masks={'dish pred': 1}

In [ ]:
a = ''
b = a.split(',').remove('')
# b.remove('')
print(b)


In [ ]:
# 定义一个字符串，包含函数代码
func_str = "def add(x, y):\n    return x + y"

# 使用 eval() 函数将字符串转换为函数
add_func = eval(func_str)

# 调用函数
result = add_func(2, 3)

# 输出结果
print(result) # 输出 5


In [3]:
import pandas as pd
from create_dataset import Food172


data_prefix = 'sub_50_'
sub_labels = []
dataset_path = '/www/datasets/Food172'
num_classes = 172

food172_train_set = Food172(root=dataset_path, prefix=data_prefix, 
                            sub_labels=sub_labels, mode='trainval',
                            augmentation=False)

# 根据 train set 构建知识图谱
multiclass_label = food172_train_set.multiclass # food标签 [1, 0, 2, ......, 171, 171, 171]
multilabel_label = food172_train_set.multilabel # 食材标签 [[0, 1, 1, ..., 0], [0, 1, 1, ..., 1], [0, 1, 1, ..., 1], ..., [0, 1, 1, ..., 1]]

assert len(multiclass_label) == len(multilabel_label)

pd_multiclass_label = pd.DataFrame(multiclass_label)
pd_multilabel_label = pd.DataFrame(multilabel_label)


In [4]:
import pandas as pd

# 假设你有一个 DataFrame df，大小为 (516, 1)
# 创建一个示例 DataFrame
data = {'column_name': [0, 2, 1, 3, 2, 0]}
df = pd.DataFrame(data)

# 查找值为 i 的索引
i = 2
indices = df[df['column_name'] == i].index

print(df['column_name'][1])

2


In [3]:
import torch
import torch.nn.functional as F

def find_most_similar_vector(a, b):
    # 计算 a 和 b 之间的余弦相似度
    similarity_scores = F.cosine_similarity(a, b.unsqueeze(0), dim=1)
    # 找到最相似向量的索引
    most_similar_index = torch.argmax(similarity_scores)
    # 获取相似度值
    similarity_value = similarity_scores[most_similar_index]

    return most_similar_index.item(), similarity_value.item()

# 示例数据
a = torch.rand(111, 227)
b = torch.rand(227)

# 查找最相似向量的索引和相似度值
most_similar_index, similarity_value = find_most_similar_vector(a, b)
print("Most similar index:", most_similar_index)
print("Similarity value:", similarity_value)


Most similar index: 94
Similarity value: 0.7931146621704102


In [33]:
import numpy as np

def insert_elements_interval(arr):
    # 计算后半部分的起始索引
    mid_index = len(arr) // 2
    print(np.arange(mid_index, len(arr)), arr[mid_index:])

    # 将后半部分的元素间隔插入前半部分
    result = np.insert(arr[:mid_index], np.arange(0, len(arr[mid_index:])), arr[mid_index:])

    return result

def replace_with_percentage(data, percentage):
    unique_elements = np.unique(data)
    
    for element in unique_elements:
        indices = np.where(data == element)[0]
        sorted_indices = insert_elements_interval(np.sort(indices))
        print('sorted_indices:', sorted_indices)
        # 将 sorted_indices 中的 第 len(sorted_indices)/2 个元素放在第 0 个元素后面，
        num_to_replace = int(len(indices) * percentage)
        indices_to_replace = sorted_indices[:num_to_replace]
        data[indices_to_replace] = -1

# 示例数据
data = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

# 复制一份数据，保留原始数据不受影响
a_copy = np.array(data)

# 将各种类的元素的 20% 替换为 -1
percentage = 0.5
replace_with_percentage(a_copy, percentage)

print(a_copy)


[2 3 4] [2 3 4]
sorted_indices: [2 0 3 1 4]
[3 4 5] [ 8  9 10]
sorted_indices: [ 8  5  9  6 10  7]
[ 6  7  8  9 10 11] [17 18 19 20 21 22]
sorted_indices: [17 11 18 12 19 13 20 14 21 15 22 16]
[-1  0 -1  0  0 -1  1  1 -1 -1  1 -1 -1 -1  2  2  2 -1 -1 -1  2  2  2]
